<a href="https://colab.research.google.com/github/hiden0/AlgoritmosOptimizacion/blob/master/Javier_Berjoyo_Trabajo_Pr%C3%A1ctico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Javier Berjoyo Madera <br>
Url: https://github.com/hiden0/AlgoritmosOptimizacion/blob/master/Javier_Berjoyo_Trabajo_Pr%C3%A1ctico.ipynb<br>
Google Colab: https://colab.research.google.com/drive/1c8WJFmuGdmO8cWfNR4GLahBHb1q_rXzp?usp=sharing <br>
Problema:
Sesiones de doblaje


Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

Los datos son:

- Número de actores: 10 <br>
- Número de tomas : 30  <br>
- Actores/Tomas : https://bit.ly/36D8IuK <br>



....







                                        

### Función objetivo

El problema consiste en asignar escenas (asociadas a actores) a días de grabación de manera que se minimice el total de actores-día utilizados. De tal forma que la función objetivo sería:
$$
Minimizar Z = \sum_{i=1}^{D} \sum_{j=1}^{A} X_{ij}
$$

En este caso, $D$ estaría asociado al número de días, $A$ al número de actores y la variable $X_{ij}$ sería el valor binario que indica si el actor $j$ acude el día $i$ al set de grabación.

### Restricciones del problema

El problema cuenta con ciertas restricciones que se deben tener en cuenta. Como el número de días de grabación y el número de tomas por día:
$$ D ≤ 6 $$
$$ T/D ≤ 6 $$

Siendo $T$ el número de tomas.<br>
También hay que acotar los valores de la variable $X_{ij}$:
$$X_{ij} \in \{0, 1\}$$

Por último, todas las tomas deben ser asignadas a uno de los días:
$$\sum_{j=1}^{D} X_{ij} = 1 \quad \text{para todo } i \in \{1,2,\dots,30\}.$$

#Análisis


### Orden de complejidad
El orden de complejidad de este problema dependerá de cómo se aborde y resuelva. Aquí hay algunas consideraciones:

1. **Enumeración de todas las combinaciones posibles**: Si intentamos enumerar todas las posibles combinaciones de tomas distribuidas en días, el número total de combinaciones posibles es enorme. Este enfoque tendría un orden de complejidad exponencial.

2. **Enfoque heurístico o algoritmo aproximado**: Si se utiliza un enfoque heurístico o un algoritmo aproximado para encontrar una solución que se acerque a la óptima, el orden de complejidad puede ser menor. Por ejemplo, se podrían utilizar técnicas de búsqueda local o algoritmos genéticos para explorar el espacio de soluciones de manera más eficiente. Sin embargo, la calidad de la solución obtenida puede no ser óptima.

### Espacio de soluciones


Si tenemos en cuenta las restricciones (6 tomas diarias) el número total de posibilidades se calcularía como: $$\left(\binom{n}{0} + \binom{n}{1} + ... + \binom{n}{6} \right) n! $$  
Lo cual da un total de $2.04 \cdot 10^{38}$ posibles soluciones.


#Diseño
Finalmente se va a hacer uso de algoritmos voraces para la optimización de este problema. Se modificarán los criterios de éstos para ver cual de ellos ofrece los mejores resultados.

In [ ]:
# Datos del problema
num_actores = 10
num_tomas = 30
num_dias = 6
dias = range(num_dias)
actores = range(num_actores)
tomas = range(num_tomas)

# Matriz de actores/tomas
actores_tomas = [
    [1,1,1,1,1,0,0,0,0,0],
    [0,0,1,1,1,0,0,0,0,0],
    [0,1,0,0,1,0,1,0,0,0],
    [1,1,0,0,0,0,1,1,0,0],
    [0,1,0,1,0,0,0,1,0,0],
    [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,1,0],
    [1,1,1,0,1,0,0,1,0,0],
    [1,1,1,1,0,1,0,0,0,0],
    [1,0,0,1,1,0,0,0,0,0],
    [1,0,1,0,0,1,0,0,0,0],
    [1,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [1,0,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,0,1,0,0],
    [1,1,1,1,0,0,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,1],
    [1,0,1,0,1,0,0,0,1,0],
    [0,0,0,1,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0],
    [1,0,0,0,1,1,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0]
]


### Criterio voraz 1
El primer criterio que seleccionaremos para el algoritmo voraz será el de seleccionar por orden las tomas con más actores en un mismo día.

In [ ]:
# Inicializamos un diccionario para almacenar las asignaciones de tomas por día
asignaciones_por_dia = {dia: [] for dia in range(num_dias)}

# Ordenamos las tomas por el número de actores requeridos
tomas_ordenadas = sorted(tomas, key=lambda i: sum(actores_tomas[i]),reverse=True)

# Iteramos sobre las tomas ordenadas
dia = 0
for i in tomas_ordenadas:

    # Verificamos si el número total de tomas en ese día no excede el límite de 6
    if len(asignaciones_por_dia[dia]) <6:
        # Asignamos la toma al día con menos tomas asignadas
        asignaciones_por_dia[dia].append(i)
    else:
      dia+=1
# Imprimimos las asignaciones por día
for dia, tomas_asignadas in asignaciones_por_dia.items():
    print(f"Día {dia+1}: Tomas asignadas {tomas_asignadas}")

Día 1: Tomas asignadas [0, 10, 11, 3, 5, 6]
Día 2: Tomas asignadas [19, 21, 24, 25, 1, 2]
Día 3: Tomas asignadas [7, 8, 12, 13, 14, 28]
Día 4: Tomas asignadas [16, 17, 18, 20, 22, 23]
Día 5: Tomas asignadas [27, 29]
Día 6: Tomas asignadas []


Comprobamos el coste:


In [ ]:
import numpy as np
def comprobar_coste(asignaciones_por_dia):
  actores_dia=0
  for dia, tomas_asignadas in asignaciones_por_dia.items():
      tomas_dia=[]
      for toma in tomas_asignadas:
        #print(toma)
        #print(actores_tomas[toma])
        tomas_dia.append(actores_tomas[toma])
      sumas_por_columna = np.sum(tomas_dia, axis=0)
      columnas_con_unos = np.sum(sumas_por_columna > 0)
      print(f"Numero de actores del día {dia}: {columnas_con_unos}")
      actores_dia+=columnas_con_unos

  print(f'Numero total de actores en el conjunto de dias: {actores_dia}')
  return actores_dia

In [ ]:
coste = comprobar_coste(asignaciones_por_dia)

Numero de actores del día 0: 8
Numero de actores del día 1: 8
Numero de actores del día 2: 7
Numero de actores del día 3: 4
Numero de actores del día 4: 2
Numero de actores del día 5: 0
Numero total de actores en el conjunto de dias: 29


### Criterio voraz 2
Cambiaremos el criterio, utilizando ahora como criterio la toma que más actores comparta con la seleccionada anteriormente, hasta rellenar las 6 tomas diarias.

In [ ]:
# Inicializamos un diccionario para almacenar las asignaciones de tomas por día
asignaciones_por_dia = {dia: [] for dia in range(num_dias)}

# Ordenamos las tomas por el número de actores requeridos
tomas_ordenadas = sorted(tomas, key=lambda i: sum(actores_tomas[i]),reverse=True)
print(tomas_ordenadas)

dias = range(6)

for dia in dias:
    #añado la toma disponible con mas participantes, la asigno como referencia y la borro del array original
    if len(tomas_ordenadas)==0:
      break
    asignaciones_por_dia[dia].append(tomas_ordenadas[0])
    toma_referencia = tomas_ordenadas[0]
    tomas_ordenadas = np.delete(tomas_ordenadas, 0, axis=0)
    for i in range(5):
        # Calcular la suma de elementos coincidentes entre la fila de referencia y todas las demás filas
        best_coincidencias=0
        for toma in tomas_ordenadas:
          coincidencias = np.sum(np.array(actores_tomas[toma_referencia]) & np.array(actores_tomas[toma]))
          if coincidencias > best_coincidencias:
            best_coincidencias = coincidencias
            best_toma = toma
        toma_referencia=best_toma
        asignaciones_por_dia[dia].append(toma_referencia)
        tomas_ordenadas=tomas_ordenadas[tomas_ordenadas != toma_referencia]




[0, 10, 11, 3, 5, 6, 9, 19, 21, 24, 25, 1, 2, 4, 7, 8, 12, 13, 14, 28, 15, 16, 17, 18, 20, 22, 23, 26, 27, 29]


In [ ]:
print(asignaciones_por_dia)
coste = comprobar_coste(asignaciones_por_dia)

{0: [0, 10, 11, 21, 5, 6], 1: [3, 14, 9, 7, 24, 8], 2: [19, 25, 1, 12, 28, 13], 3: [2, 4, 15, 26, 27, 29], 4: [16, 18, 22, 17, 23, 20], 5: []}
Numero de actores del día 0: 7
Numero de actores del día 1: 8
Numero de actores del día 2: 6
Numero de actores del día 3: 7
Numero de actores del día 4: 4
Numero de actores del día 5: 0
Numero total de actores en el conjunto de dias: 32


### Criterio voraz 3
La optimización, más allá de haber mejorado, por lo que hay que reforzar el criterio. La comparativa de coincidencias pasará de hacer con la última toma seleccionada a hacerse con todas las tomas anteriormente seleccionadas ese dia.

In [ ]:
# Inicializamos un diccionario para almacenar las asignaciones de tomas por día
asignaciones_por_dia = {dia: [] for dia in range(num_dias)}

# Ordenamos las tomas por el número de actores requeridos
tomas_ordenadas = sorted(tomas, key=lambda i: sum(actores_tomas[i]),reverse=True)
print(tomas_ordenadas)
actores_tomas = np.array(actores_tomas)
dias = range(6)

for dia in dias:
    #añado la toma disponible con mas participantes, la asigno como referencia y la borro del array original
    if len(tomas_ordenadas)==0:
      break
    asignaciones_por_dia[dia].append(tomas_ordenadas[0])
    toma_referencia = tomas_ordenadas[0]
    tomas_ordenadas = np.delete(tomas_ordenadas, 0, axis=0)
    for i in range(5):
        # Calcular la suma de elementos coincidentes entre la fila de referencia y todas las demás filas
        best_coincidencias=0
        for toma in tomas_ordenadas:
          coincidencias = np.sum(np.all(np.array(actores_tomas[asignaciones_por_dia[dia]]) & np.array(actores_tomas[toma]), axis=0))
          if coincidencias > best_coincidencias:
            best_coincidencias = coincidencias
            best_toma = toma
        toma_referencia=best_toma
        asignaciones_por_dia[dia].append(toma_referencia)
        tomas_ordenadas=tomas_ordenadas[tomas_ordenadas != toma_referencia]


[0, 10, 11, 3, 5, 6, 9, 19, 21, 24, 25, 1, 2, 4, 7, 8, 12, 13, 14, 28, 15, 16, 17, 18, 20, 22, 23, 26, 27, 29]


In [ ]:
print(asignaciones_por_dia)
coste = comprobar_coste(asignaciones_por_dia)

{0: [0, 10, 11, 21, 3, 5], 1: [6, 19, 12, 24, 8, 27], 2: [9, 7, 13, 28, 25, 14], 3: [1, 26, 2, 2, 2, 2], 4: [4, 15, 29, 29, 29, 29], 5: [16, 18, 22, 17, 23, 23]}
Numero de actores del día 0: 8
Numero de actores del día 1: 6
Numero de actores del día 2: 7
Numero de actores del día 3: 5
Numero de actores del día 4: 5
Numero de actores del día 5: 3
Numero total de actores en el conjunto de dias: 34


### Criterio voraz 4
El criterio resulta demasiado restrictivo, por lo que los resultados son peores. Se cambia para utilizar la suma de coincidencias entre tomas en lugar de la coincidencia general.

In [ ]:
# Inicializamos un diccionario para almacenar las asignaciones de tomas por día
asignaciones_por_dia = {dia: [] for dia in range(num_dias)}

# Ordenamos las tomas por el número de actores requeridos
tomas_ordenadas = sorted(tomas, key=lambda i: sum(actores_tomas[i]),reverse=True)
print(tomas_ordenadas)
actores_tomas = np.array(actores_tomas)
dias = range(6)

for dia in dias:
    #añado la toma disponible con mas participantes, la asigno como referencia y la borro del array original
    if len(tomas_ordenadas)==0:
      break
    asignaciones_por_dia[dia].append(tomas_ordenadas[0])
    toma_referencia = tomas_ordenadas[0]
    tomas_ordenadas = np.delete(tomas_ordenadas, 0, axis=0)
    for i in range(5):
        # Calcular la suma de elementos coincidentes entre la fila de referencia y todas las demás filas
        best_coincidencias=0
        for toma in tomas_ordenadas:
          coincidencias = np.sum(np.array(actores_tomas[asignaciones_por_dia[dia]]) & np.array(actores_tomas[toma]))
          if coincidencias > best_coincidencias:
            best_coincidencias = coincidencias
            best_toma = toma
        toma_referencia=best_toma
        asignaciones_por_dia[dia].append(toma_referencia)
        tomas_ordenadas=tomas_ordenadas[tomas_ordenadas != toma_referencia]


[0, 10, 11, 3, 5, 6, 9, 19, 21, 24, 25, 1, 2, 4, 7, 8, 12, 13, 14, 28, 15, 16, 17, 18, 20, 22, 23, 26, 27, 29]


In [ ]:
print(asignaciones_por_dia)
coste = comprobar_coste(asignaciones_por_dia)

{0: [0, 10, 11, 21, 5, 6], 1: [3, 14, 9, 7, 24, 8], 2: [19, 25, 1, 12, 28, 13], 3: [2, 4, 26, 15, 27, 29], 4: [16, 18, 22, 17, 23, 20], 5: []}
Numero de actores del día 0: 7
Numero de actores del día 1: 8
Numero de actores del día 2: 6
Numero de actores del día 3: 7
Numero de actores del día 4: 4
Numero de actores del día 5: 0
Numero total de actores en el conjunto de dias: 32


Los resultados son los mismos que con el criterio 2.